In [ ]:
#10 M events were generated at PX positon including the isotopes 'Th228','Ra224','Rn220','Po216','Pb212','Bi212','Tl208'
#skipEThreshold: 0.1
#Each isotopes 40 seeds;each simulation: 250K events
# 38 failed for Th228 and 27 failed for Po216
#Total root files in g4 215
#Total root files in recon 211

In [ ]:
#libraries
import uproot
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import rcParams
import seaborn as sns
import scienceplots
sns.set(font_scale=1.0)
sns.set_style('white')

# DATA FILES

In [ ]:
#cuts
apply_filter='passed_z_thresh & passed_xy_thresh & (n_x_ch_abovenoise>0) & (n_y_ch_abovenoise>0) & (m_nOPCal< (1.064*m_nQ+703)) & (m_nOPCal> (0.644*m_nQ-2411)) & (~NESTBugFound) & (m_DNNvalue>0.85) & (standoff > 100)'


dec102020_filter='(standoff>100) & passed_xy_thresh & passed_z_thresh & (n_x_ch_abovenoise > 0) & (n_y_ch_abovenoise > 0) & (m_nOPCal < (1.077 * m_nQ + 313)) & (m_nOPCal > (0.597 * m_nQ - 216)) & ~NESTBugFound & ~NearAnodeBugFound & m_DNNvalue>0.85)'
ja_filter='(standoff>100) & m_DNNvalue>0.85 & passed_xy_thresh & passed_z_thresh & (n_x_ch_abovenoise > 0) & (n_y_ch_abovenoise > 0) & (m_nOPCal < (1.077 * m_nQ + 313)) & (m_nOPCal > (0.597 * m_nQ - 216))& (abs(energy-2614)<10)'# & ~NESTBugFound & ~NearAnodeBugFound' & (abs(energy-2614)<10)'
s_filter='''
            standoff>20   &\
            m_DNNvalue>0  &\
            energy>0      &\
            energy<3000   &\
            passed_z_thresh &\
            passed_xy_thresh &\
            (n_x_ch_abovenoise>0) &\
            (n_y_ch_abovenoise>0) &\
            (m_nOPCal< (1.064*m_nQ+703))&\
            (m_nOPCal> (0.644*m_nQ-2411))&\
            ~NESTBugFound &\
            ~NearAnodeBugFound       
         '''

In [ ]:
file='/home/thakur/slac_data/10Mth228/10Mth228_Th228_pos648.5_0_-1022.6.root'

In [ ]:
#file dataframe
f=uproot.open(file+':tree')
df=f.arrays(f.keys(),library='pd')

In [ ]:
df

In [ ]:
#columns
df.columns

In [ ]:
#application of s_filter
df_s=df.query(s_filter)

In [ ]:
df_s

In [ ]:
#isotope coutns before the cut
raw_counts=pd.DataFrame(df.isotope.value_counts().reset_index().values,columns=['isotopes','value'])\
.sort_values(by=['isotopes'])\
.reset_index(drop=True)
raw_counts

In [ ]:
#isotope couns after eh cut
cut_counts=pd.DataFrame(df_s.isotope.value_counts().reset_index().values,columns=['isotopes','values'])\
.sort_values(by=['isotopes'])\
.reset_index(drop=True)

cut_counts

In [ ]:
#isotopes counts
df_s.isotope.value_counts()

In [ ]:
isotopes=['Th228','Ra224','Rn220','Po216','Pb212','Bi212','Tl208']

In [ ]:
positions


In [ ]:
files=['s38_'+iso+'_'+pos+'.root' for iso in isotopes for pos in positions]

In [ ]:
files

In [ ]:
file=files[0]
iso_pos=file.split(".")[0].split("_")[1:]
iso_pos[0],iso_pos[1]

In [ ]:
file=files[7]

def return_values(file,cut):
    iso_pos=file.split(".")[0].split("_")[1:]
    iso=iso_pos[0];pos=iso_pos[1]

# if 'Bi212' in file:
#     print("It is Bi212")
# else:
#     print("It is Tl208")

    f=uproot.open(file+':tree')
    df=f.arrays(f.keys(),library='pd')
    raw_counts=df.shape[0]
    filtered_counts=df.query(cut).shape[0]
#     print(f"""
# filename       :       {file}
# isotope        :       {iso}
# position       :       {pos}
# raw_counts     :       {raw_counts}
# filtered_counts:       {filtered_counts}
# """)
    return [file,iso,pos,raw_counts,filtered_counts]

In [ ]:
def get_df(cut):
    print(f"CUT:\n{cut}\n")
    all_info=[]
    for file in files:
        all_info.append(return_values(file,cut))
    df_all=pd.DataFrame(all_info,columns=['file','isotope','position','g4tree_counts','cut_counts'])
    print(35*'--')
    print(f"Complete DataFrame:\n{df_all.to_string()}")
    print(35*'--')
    print("raw and cut counts\n")
    print(df_all[['g4tree_counts','cut_counts']].sum())
    print(35*'--')
    df_bi=df_all[df_all['isotope']=='Bi212'][['g4tree_counts','cut_counts']]
    df_tl=df_all[df_all['isotope']=='Tl208'][['g4tree_counts','cut_counts']]
    print(f"\nBi212 DataFrame:\n{df_bi.to_string()}")
    print("\nraw and cut counts for Bi212\n")
    print(f"{df_bi.sum()}")
    print(35*'--')
    
    print(f"\nTl208 DataFrame:\n{df_tl.to_string()}")
    print("\nraw and cut counts for Tl208\n")
    print(f"{df_tl.sum()}")
    print(35*'--')
    #return df_all,df_bi,df_tl

In [ ]:
#filter 1
j_df=get_df(jason_filter)
j_df

In [ ]:
#filter 2
d_df=get_df(dec102020_filter)
d_df

In [ ]:
# dataframe
df_all=pd.DataFrame(all_info,columns=['file','isotope','position','raw_counts','filtered_counts'])

In [ ]:
df_all

In [ ]:
#all counts
df_all[['raw_counts','filtered_counts']].sum()

In [ ]:
# Bi212
df_bi=df_all[df_all['isotope']=='Bi212'][['raw_counts','filtered_counts']]
df_bi

In [ ]:
# Bi212 sum
df_bi.sum()

In [ ]:
# Tl208
df_tl=df_all[df_all['isotope']=='Tl208'][['raw_counts','filtered_counts']]
df_tl

In [ ]:
#sum of counts
df_tl.sum()

In [ ]:
df.isotope.value_counts

In [ ]:
df_filtered=df.query(jason_filter)

In [ ]:
df_filtered